- What are the initial velocities? (Can be defined in restart file), set this to the correct intial temperature and set parameter for rescaling to zero
- Why are the positions negative after first iteration (MIC?) ? maybe displacement from the initial positions?

- run initial MD for 1 step with PROFESS
- store geometry, velocity, forces

for NSTEPS:
    - use geometry and velocity from previous run as restart files 
    - run MD for 1 step, store geometry, velocity, forces
    
in ase use geometry, velocity and forces

## Simple MD without thermostat

In [1]:
from ase import Atoms
import numpy as np
from ase.calculators.general import Calculator
from ase.md.verlet import VelocityVerlet
from ase import units
import parse_profess_out as ppo

In [2]:
# velocities from Bohr/aut (PROFESS) -> Ang/ (Ang sqrt(aum/eV)) (ASE)

# Bohr -> Ang = length_in_Bohr*units.Bohr
# aut -> Ang sqrt(aum/eV)
aut2fs = 2.41888433e-2 # go from aut -> fs
aut2aset = aut2fs*units.fs # from aut -> aset (ase time unit)

au_vel2ase_vel = units.Bohr/aut2aset # conversion of velocities in a.u. to native ase units

In [3]:
au_vel2ase_vel

222.71803270527124

In [4]:
def initialize(pos_file, vel_file, for_file, cell_param):
    """
    generate atoms object with:
        - correct positions
        - cell paramters
        - set initial velocities
    set calculator
    """
    positions = ppo.parse_ion_file(pos_file)
    
    velocities = np.array(ppo.parse_velocity_file(vel_file))*au_vel2ase_vel
    
    atoms = Atoms('Al4', positions = positions, cell = cell_param, pbc = True)
    atoms.set_velocities(velocities)
    atoms.set_calculator(OFDFT_calculator(for_file))
    return(atoms)
    
class OFDFT_calculator(Calculator):
    name = 'OFDFT_calculator'
    implemented_properties = ['forces']

    def __init__(self, for_file):
        self.forces = np.array(ppo.parse_force_file(for_file))
#         self.nAtoms = atoms.get_number_of_atoms()

    def get_forces(self, atoms=None):
        # call ofdft code via os subprocesses
        # input new coords, new density
        return(self.forces)

In [6]:
pos_file = '/home/misa/git_repositories/PROFESS/test/MD_NVE/saved/ion_step0.dat'
vel_file = '/home/misa/git_repositories/PROFESS/test/MD_NVE/saved/vel_step0.dat'
for_file = '/home/misa/git_repositories/PROFESS/test/MD_NVE/saved/force_step0.dat'
cell_param = [[3.97, 0, 0], [0, 3.97, 0], [0, 0, 3.97]]


atoms = initialize(pos_file, vel_file, for_file, cell_param)
# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = VelocityVerlet(atoms, 1*units.fs)  # 1 fs time step.

# run only for one step
f=dyn.run(1)

print(atoms.get_positions())
print(atoms.get_velocities())

[[-1.00149331 -1.10260569 -0.98778055]
 [ 0.99624208  1.28811246 -0.99196346]
 [-0.99497734  0.9009717   0.97709232]
 [ 1.0002287  -1.08647898  1.00265184]]
[[-0.0456017  -0.05154583  0.02395716]
 [ 0.01891601 -0.01824453  0.00276711]
 [-0.01255187  0.03940702 -0.07828639]
 [ 0.03923963  0.03037707  0.05156447]]


In [7]:
length = 3.97
new_atoms = []
for atom in atoms.get_positions():
    tmp = []
    for i,c in enumerate(atom):
        
        if c < 0:
            tmp.append(c + length)
        else:
            tmp.append(c)
    new_atoms.append(tmp)
new_atoms

[[2.9685066882591933, 2.8673943134403506, 2.9822194495411742],
 [0.9962420840664423, 1.2881124636073888, 2.9780365420083412],
 [2.975022659972352, 0.9009717021857254, 0.9770923190428329],
 [1.0002287035256463, 2.8835210196277226, 1.0026518428375466]]

In [9]:
pos_file1 = '/home/misa/git_repositories/PROFESS/test/MD_NVE/saved/ion_step1.dat'
pos1 = ppo.parse_ion_file(pos_file1)
length = 3.97
new_atoms = []
for atom in pos1:
    tmp = []
    for i,c in enumerate(atom):
        
        if c < 0:
            tmp.append(c + length)
        else:
            tmp.append(c)
    new_atoms.append(tmp)
new_atoms

[[0.983506643, 0.882395148, 0.997219443],
 [2.9812418980000004, 3.2731123610000004, 0.99303638],
 [0.990022802, 2.88597048, 2.9620923900000005],
 [2.985228674, 0.898521916, 2.9876517970000003]]

### MD run

In [ ]:
# create atoms object
# .ion-file
at1 = [0,0,0]
at2 = [0.5,0.5,0]
at3 = [0,0.5,0.5]
at4 = [0.5,0,0.5]
cell1 = [3.9691, 0, 0]
cell2 = [0, 3.9691, 0]
cell3 = [0, 0, 3.9691]

atoms = Atoms('Al4',
             scaled_positions=[at1,at2, at3, at4],
             cell=[cell1, cell2, cell3],
             pbc=True)
forces_nuclei = np.array([[0.0,0,0],[0,0,0],[0,0,0],[0,0,0]])

# set the forces from the OFDFT code
# improvement: call ofdft in this function?
# get_forces reads forces from file
atoms.set_calculator(OFDFT_calculator(forces_nuclei))

In [ ]:
atoms.get_positions()

In [ ]:
# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = VelocityVerlet(atoms, 5 * units.fs)  # 5 fs time step.

# run only for one step
f=dyn.run(1)

# get new positions for force calculations
atoms.get_positions()

In [ ]:
atoms

In [ ]:
pos

files for ofdft:
- job.inpt (stays the same during CPMD)
- job.ion (structure: update position, lattice vectors?)
- density (update from CPMD)
- al.lda.recpot (pseudopotentials: stay the same during CPMD)

Which code to do APDFT calculations? (must support crystal structures)
Are local ofdft pseudopotentials compatible with other non-local pseudopotentials?

How to do MD runs in OFDFT for comparison?

How to run NVE, What is NVT coupled with Nose-Hoover?
Why are the positions negative, so close togethcer and different from initial structure?



- get forces, velocities from ofdft during python runtime
- propagate nuclei


            